# Secretary Module - Interactive Testing

Just run each cell in order and input your data when prompted!

## Setup - Clone Repo & Navigate

**Run this cell FIRST:**

In [ ]:
# Setup - Clone repo and navigate to secretary module
import os

if not os.path.exists('/content/ESTA-Final'):
    print("Cloning repository...")
    !git clone https://github.com/JPmarsxxxi/ESTA-Final.git
    print("✅ Repository cloned!")
else:
    print("Repository already exists, pulling latest changes...")
    !cd /content/ESTA-Final && git pull origin claude/secretary-module-orchestration-LPq69

print("\nChecking out branch...")
!cd /content/ESTA-Final && git checkout claude/secretary-module-orchestration-LPq69

print("\nNavigating to secretary module...")
%cd /content/ESTA-Final/workers/secretary

print("\n✅ Setup complete! Current directory:")
!pwd
print("\nFiles in this directory:")
!ls -la

In [ ]:
# Install Ollama in Colab
print("Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

print("\n✅ Ollama installed!")

# Start Ollama server in background
print("\nStarting Ollama server...")
import subprocess
import time

# Start Ollama serve in background
ollama_process = subprocess.Popen(
    ['ollama', 'serve'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait a bit for server to start
time.sleep(3)

print("✅ Ollama server started!")

# Pull Mistral model
print("\nPulling Mistral model (this may take 2-3 minutes)...")
!ollama pull mistral

print("\n✅ Mistral model ready!")

# Verify it's working
print("\nVerifying Ollama is responding...")
!curl -s http://localhost:11434/api/tags

print("\n✅ Ollama setup complete!")

## Install Ollama & Dependencies

**This will take 3-5 minutes on first run:**
- Install Ollama
- Start Ollama server
- Pull Mistral 7B model (~4GB download)
- Install Python packages

**Run this cell and wait for "✅ Ollama setup complete!"**

In [ ]:
# Install Python dependencies
print("Installing Python dependencies...")
!pip install -q requests
print("✅ Python dependencies installed!")

## Import Secretary Module

**Run this after setup:**

In [ ]:
# Import the Secretary module
from secretary import Secretary, create_full_workflow
from tool_registry import AVAILABLE_TOOLS, REAL_SCRIPTWRITER_AVAILABLE
import json

print("✅ Secretary module loaded successfully!")
print(f"\n🔧 ScriptWriter Status: {'REAL ✅' if REAL_SCRIPTWRITER_AVAILABLE else 'MOCK ⚠️'}")

if not REAL_SCRIPTWRITER_AVAILABLE:
    print("\n⚠️  Real ScriptWriter not available - using mock version")
    print("To use real ScriptWriter, ensure:")
    print("  1. Ollama is running: ollama serve")
    print("  2. Mistral model installed: ollama pull mistral")
    print("  3. All dependencies installed")
    print("\nCheck import errors by running:")
    print("  from scriptwriter import ScriptWriter")
else:
    print("✅ Ready to generate real scripts with Ollama + LangSearch!")

## Interactive Input Collection

**Run this cell - it will prompt you for each input:**

In [ ]:
print("=" * 60)
print("VIDEO REQUIREMENTS - Answer each question:")
print("=" * 60)

topic = input("\n1. What's your video topic? (min 5 chars): ")

print("\n2. Choose style:")
print("   Options: funny, documentary, serious, graphic-heavy, tutorial")
print("   Or type your own custom style (min 3 chars)")
style = input("   Your choice: ")

print("\n3. Duration (e.g., '5 minutes', '2-3 minutes', '30 seconds'):")
duration = input("   Duration: ")

print("\n4. Audio mode:")
audio = input("   Type 'generate' or 'upload': ")

print("\n5. Script mode:")
script = input("   Type 'generate' or 'upload': ")

print("\n6. Any comments? (optional, press Enter to skip)")
comments = input("   Comments: ")

# Build requirements
requirements = {
    'topic': topic,
    'style': style,
    'duration_range': duration,
    'audio_mode': audio,
    'script_mode': script,
    'comments': comments if comments else None
}

print("\n" + "=" * 60)
print("YOUR REQUIREMENTS:")
print("=" * 60)
print(json.dumps(requirements, indent=2))

## Validate Requirements

Check if your inputs are valid:

In [ ]:
sec = Secretary()
is_valid, errors = sec.validate_inputs(requirements)

print("\n" + "=" * 60)
print("VALIDATION RESULT")
print("=" * 60)

if is_valid:
    print("✅ ALL INPUTS VALID!")
    sec.set_requirements(requirements)
else:
    print("❌ VALIDATION FAILED!\n")
    print("Errors:")
    for error in errors:
        print(f"  - {error}")
    print("\n⚠️  Fix the errors above and run the input cell again!")

## Worker 1: ScriptWriter

Generate a script from your requirements:

In [ ]:
print("🚀 Calling REAL ScriptWriter (Ollama + LangSearch)...")
print("This will take 30-90 seconds depending on your system.")
print("Watch the logs below to see each step!\n")

result = sec.call_tool('scriptwriter', sec.requirements)

if result['success']:
    print("\n" + "="*80)
    print("✅ SCRIPT GENERATION SUCCESSFUL!")
    print("="*80)

    # Show metadata first
    metadata = result.get('metadata', {})
    print(f"\n📊 GENERATION STATS:")
    print(f"   Model: {metadata.get('model_used', 'N/A')}")
    print(f"   Target words: {metadata.get('target_words', 'N/A')}")
    print(f"   Actual words: {metadata.get('actual_words', 'N/A')}")
    print(f"   Timestamp: {metadata.get('timestamp', 'N/A')}")

    # Show research sources
    outputs = result['outputs']
    sources = outputs.get('research_sources', [])

    if sources:
        print(f"\n📚 RESEARCH SOURCES ({len(sources)} found):")
        for i, source in enumerate(sources[:5], 1):
            print(f"   {i}. {source.get('title', 'Untitled')}")
            if source.get('url'):
                print(f"      URL: {source['url']}")
            if source.get('snippet'):
                print(f"      → {source['snippet'][:100]}...")

    # Show talking points if available
    if 'talking_points_count' in outputs:
        print(f"\n💡 TALKING POINTS: {outputs['talking_points_count']} main points generated")

    # Show structure breakdown
    structure = outputs.get('structure', {})
    if structure:
        print(f"\n📝 SCRIPT STRUCTURE:")
        print(f"   Hook: {len(structure.get('hook', '').split())} words")
        print(f"   Body: {len(structure.get('body', []))} paragraphs")
        print(f"   CTA: {len(structure.get('cta', '').split())} words")

    # Show estimated duration
    est_duration = outputs.get('estimated_duration_minutes', 0)
    print(f"\n⏱️  ESTIMATED DURATION: {est_duration} minutes")
    print(f"   Total words: {outputs.get('word_count', 0)}")

    # Show the full script
    print("\n" + "="*80)
    print("📜 GENERATED SCRIPT:")
    print("="*80)
    print(outputs['script'])
    print("="*80)

    # Save for next steps
    script = outputs['script']

    print(f"\n✅ Script saved! Ready for next worker (Audio Agent)")

else:
    print("="*80)
    print("❌ SCRIPT GENERATION FAILED!")
    print("="*80)
    print(f"Error: {result.get('error', 'Unknown error')}")
    print("\nCheck the logs above for details.")
    print("\nCommon issues:")
    print("  - Ollama not running (run: ollama serve)")
    print("  - Mistral model not installed (run: ollama pull mistral)")
    print("  - LangSearch API key invalid")
    print("  - Network connection issues")

## Worker 2: Audio Agent

Generate audio from the script:

In [ ]:
print("Calling Audio Agent...\n")

result = sec.call_tool('audio_agent', {'script': script})

if result['success']:
    print("✅ AUDIO GENERATED!\n")
    print(f"Audio file: {result['outputs']['audio_file']}")
    print(f"Duration: {result['outputs']['duration']} seconds")
    print(f"Sample rate: {result['outputs']['sample_rate']} Hz")
    print(f"\nTranscript segments: {len(result['outputs']['transcript_timestamps']['segments'])}")
    
    # Save for next steps
    audio_file = result['outputs']['audio_file']
    transcript = result['outputs']['transcript_timestamps']
    
    print("\nTimestamp Preview:")
    for seg in result['outputs']['transcript_timestamps']['segments'][:3]:
        print(f"  {seg['start']:.1f}s - {seg['end']:.1f}s: {seg['text'][:50]}...")
else:
    print(f"❌ FAILED: {result['error']}")

## Worker 3: LangSearch

Research terms in the script:

In [ ]:
print("Calling LangSearch...\n")

result = sec.call_tool('langsearch', {'script': script})

if result['success']:
    print("✅ RESEARCH COMPLETE!\n")
    research = result['outputs']['research_data']
    print(f"Terms found: {research['total_terms_found']}")
    print(f"Search time: {research['search_time']}s")
    
    print("\nResearched Terms:")
    for term in research['terms']:
        print(f"  - {term['term']} (relevance: {term['relevance_score']})")
        print(f"    Context: {term['context'][:60]}...")
    
    # Save for next steps
    research_data = research
else:
    print(f"❌ FAILED: {result['error']}")

## Worker 4: BrainBox

Create video plan:

In [ ]:
print("Calling BrainBox...\n")

result = sec.call_tool('brainbox', {
    'script': script,
    'transcript_timestamps': transcript,
    'research_data': research_data,
    'requirements': sec.requirements
})

if result['success']:
    print("✅ VIDEO PLAN CREATED!\n")
    plan = result['outputs']['video_plan']
    print(f"Total duration: {plan['total_duration']}s")
    print(f"Total assets needed: {plan['total_assets']}")
    print(f"Theme: {plan['theme']}")
    
    print("\nTimeline:")
    for item in plan['timeline']:
        print(f"  {item['timestamp']}: {item['asset_type'].upper()}")
        print(f"    → {item['description']}")
    
    # Save for next steps
    video_plan = plan
else:
    print(f"❌ FAILED: {result['error']}")

## Worker 5: Asset Collector

Collect all assets:

In [ ]:
print("Calling Asset Collector...\n")

result = sec.call_tool('asset_collector', {'video_plan': video_plan})

if result['success']:
    print("✅ ASSETS COLLECTED!\n")
    manifest = result['outputs']['asset_manifest']
    print(f"Total assets: {manifest['total_collected']}")
    print(f"Assets folder: {manifest['assets_folder']}")
    
    print("\nAssets:")
    for asset in manifest['assets']:
        print(f"  [{asset['id']}] {asset['type'].upper()}")
        print(f"    File: {asset['file']}")
        print(f"    Timestamp: {asset['timestamp']}")
        print(f"    Status: {asset['status']}")
    
    # Save for next step
    assets = manifest
else:
    print(f"❌ FAILED: {result['error']}")

## Worker 6: Executor

Render final video:

In [ ]:
print("Calling Executor...\n")

result = sec.call_tool('executor', {
    'video_plan': video_plan,
    'assets': assets,
    'audio': audio_file
})

if result['success']:
    print("✅ VIDEO RENDERED!\n")
    print("=" * 60)
    print("FINAL VIDEO:")
    print("=" * 60)
    video = result['outputs']
    print(f"File: {video['video_file']}")
    print(f"Duration: {video['duration']}s")
    print(f"Resolution: {video['resolution']}")
    print(f"FPS: {video['fps']}")
    print(f"File size: {video['file_size_mb']} MB")
    print("\n🎉 COMPLETE PIPELINE SUCCESSFUL!")
else:
    print(f"❌ FAILED: {result['error']}")

## Run Full Workflow (All at Once)

Or run everything in one go:

In [ ]:
print("Running full workflow...\n")

workflow = create_full_workflow(sec.requirements)
result = sec.execute_workflow(workflow)

if result['success']:
    print("\n" + "=" * 60)
    print("✅ FULL WORKFLOW COMPLETE!")
    print("=" * 60)
    print(f"\nCompleted {result['completed_steps']}/{result['total_steps']} steps\n")
    
    for step_idx, step_data in result['results'].items():
        tool = step_data['tool']
        outputs = list(step_data['result']['outputs'].keys())
        print(f"✓ Step {step_idx + 1}: {tool} → {', '.join(outputs)}")
    
    # Final video info
    final = result['results'][5]['result']['outputs']
    print("\n" + "=" * 60)
    print(f"🎬 FINAL VIDEO: {final['video_file']}")
    print(f"   {final['duration']}s | {final['resolution']} | {final['file_size_mb']}MB")
    print("=" * 60)
else:
    print(f"\n❌ Failed at step {result['failed_step'] + 1}")
    failed = result['results'][result['failed_step']]
    print(f"Tool: {failed['tool']}")
    print(f"Error: {failed['result']['error']}")